In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings 
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
pio.templates.default = "plotly_white"
warnings.filterwarnings('ignore')

from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor


In [2]:
df = pd.read_csv('retail_price.csv')
df.head(10).T

,0,1,2,3,4,5,6,7,8,9
product_id,bed1,bed1,bed1,bed1,bed1,bed1,bed1,bed1,bed1,bed1
product_category_name,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table,bed_bath_table
month_year,01-05-2017,01-06-2017,01-07-2017,01-08-2017,01-09-2017,01-10-2017,01-11-2017,01-12-2017,01-01-2018,01-02-2018
qty,1,3,6,4,2,3,11,6,19,18
total_price,45.95,137.85,275.7,183.8,91.9,137.85,445.85,239.94,759.81,719.82
freight_price,15.1,12.933333,14.84,14.2875,15.1,15.1,15.832727,15.23,16.533684,13.749444
unit_price,45.95,45.95,45.95,45.95,45.95,45.95,40.531818,39.99,39.99,39.99
product_name_lenght,39,39,39,39,39,39,39,39,39,39
product_description_lenght,161,161,161,161,161,161,161,161,161,161
product_photos_qty,2,2,2,2,2,2,2,2,2,2


In [4]:
df.columns

Index(['product_id', 'product_category_name', 'month_year', 'qty',
       'total_price', 'freight_price', 'unit_price', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_score', 'customers', 'weekday', 'weekend', 'holiday', 'month',
       'year', 's', 'volume', 'comp_1', 'ps1', 'fp1', 'comp_2', 'ps2', 'fp2',
       'comp_3', 'ps3', 'fp3', 'lag_price'],
      dtype='object')

In [6]:
# 1. Calculate Annual Demand (D) for each product
annual_demand = df.groupby('product_id')['qty'].sum()
annual_demand

product_id
bed1          143
bed2          487
bed3          111
bed4          131
bed5          154
computers1    139
computers2    183
computers3    103
computers4    341
computers5    127
computers6    274
consoles1     142
consoles2      97
cool1         120
cool2         119
cool3         114
cool4         108
cool5          97
furniture1    169
furniture2    527
furniture3     97
furniture4    101
garden1       148
garden10      391
garden2       127
garden3       141
garden4       388
garden5       113
garden6       484
garden7       115
garden8       118
garden9       373
health1       122
health10      155
health2       195
health3       156
health4       137
health5       156
health6       150
health7       231
health8       259
health9       281
perfumery1    138
perfumery2    106
watches1      194
watches2      146
watches3      160
watches4      225
watches5      143
watches6      323
watches7      123
watches8      117
Name: qty, dtype: int64

# Product Assortment Optimization:
to optimize your product assortment, identifying high-demand categories and products to prioritize, and discontinuing low-performing items to streamline your product portfolio.This code will calculate total quantity sold, total revenue, average freight price, average product score, and average number of product photos for each product category. It will then sort the categories based on total revenue and identify top-performing and low-performing categories. You can adjust the threshold for identifying low-performing categories based on your specific criteria.


In [16]:
# Group sales data by product category and calculate total revenue and quantity sold
category_sales = df.groupby('product_category_name').agg({
    'qty': 'sum',                # total quantity sold
    'total_price': 'sum',        # total revenue generated
    'freight_price': 'mean',     # average freight price
    'product_score': 'mean',     # average product score
    'product_photos_qty': 'mean' # average number of product photos
}).reset_index()

# Calculate additional metrics
category_sales['avg_price_per_unit'] = category_sales['total_price'] / category_sales['qty']

# Sort categories by total revenue or quantity sold
category_sales_sorted = category_sales.sort_values(by='total_price', ascending=False)

# Identify top-performing categories (e.g., top 3)
top_categories = category_sales_sorted.head(3)

threshold = 10000

# Identify low-performing categories (e.g., categories with low revenue)
# You can define your criteria for identifying low-performing categories based on your business needs
low_performing_categories = category_sales_sorted[category_sales_sorted['total_price'] < threshold]  # Set threshold as needed

# Print results
print("Top-performing categories:")
print(top_categories)

print("\nLow-performing categories:")
print(low_performing_categories)

Top-performing categories:
  product_category_name   qty  total_price  freight_price  product_score  \
6         health_beauty  1842    212409.24      18.607448       4.145385   
8         watches_gifts  1431    207582.17      16.492840       3.983495   
5          garden_tools  2398    163582.54      28.458310       4.161875   

   product_photos_qty  avg_price_per_unit  
6            2.053846          115.314463  
8            2.611650          145.060915  
5            1.687500           68.216239  

Low-performing categories:
  product_category_name  qty  total_price  freight_price  product_score  \
2        consoles_games  239       5800.7      14.809415       4.063636   

   product_photos_qty  avg_price_per_unit  
2            3.545455           24.270711  


In [8]:
category_sales

,product_category_name,qty,total_price,freight_price,product_score,product_photos_qty
0,bed_bath_table,1026,95084.52,16.139718,3.908197,2.245902
1,computers_accessories,1167,142097.95,25.103741,3.985507,1.000000
2,consoles_games,239,5800.70,14.809415,4.063636,3.545455
3,cool_stuff,558,57956.30,18.975096,4.207018,1.859649
4,furniture_decor,894,56925.16,16.944617,3.979167,2.354167
5,garden_tools,2398,163582.54,28.458310,4.161875,1.687500
6,health_beauty,1842,212409.24,18.607448,4.145385,2.053846
7,perfumery,244,20312.52,14.336311,4.350000,1.500000
8,watches_gifts,1431,207582.17,16.492840,3.983495,2.611650
